
*This notebook provides the evaluation code used for benchmarking QAOA, Warm-start QAOA, a variant of warm-start QAOA, VQE, Quantum Annealing along with some classical methods such as Integer Programming, Simulated Annealing etc.. The code provided was used to generate the results presented in the paper https://arxiv.org/abs/2301.05750 .*



In [59]:
# -*- coding: utf-8 -*-
"""
@author: QUTAC Quantum

SPDX-FileCopyrightText: 2023 QUTAC

SPDX-License-Identifier: Apache-2.0

"""

'\n@author: QUTAC Quantum\n\nSPDX-FileCopyrightText: 2023 QUTAC\n\nSPDX-License-Identifier: Apache-2.0\n\n'

# Multi-Knapsack Problem

We first present the mathematical model for the multi-knapsack optimization problem. Given $N$ items and $M$ knapsacks, the aim of the problem is to assign as many items to one of the knapsacks whicle not exceeding the capacity of any knapsack. Additionally, each item in some knapsack contributes to a addedd value.

Formally, we can state the problem in the following manner. Let, $ v_j$ be the value of item $j$, $j \in \{0,1,\dots, N-1\}$, $w_j$ be the weight of time $j$, and $c_i$ be the capacity of knapsack $i$, $i\in \{0,1,\dots, M-1\}$. We can define a decision variable $x_{i,j}$, such that
$$
\begin{equation*}
    x_{i,j}=
    \begin{cases}
      1, & \text{if item $j$ is assigned to knapsack $i$}\;, \\
      0, & \text{otherwise}\;.
    \end{cases}
  \end{equation*}
$$


The multi-knapsack problem can be formulated as,

$$
\begin{equation*}
\begin{split}
\max &\sum_{i=0}^{M-1} \sum_{j=0}^{N-1} v_{i,j} \cdot x_{i,j} \\
\text{such that } & \sum_{j=0}^{N-1} w_j \cdot x_{i,j} \leq c_i, \hspace{1em} \forall i \in \{0,1,\dots, M-1\}\\
& \sum_{i=0}^{M-1} x_{i,j} \leq 1, \hspace{2em} \forall j \in \{0,1,\dots, N-1\} \\
\end{split}
\end{equation*}
$$

## QUBO Formulation

### 1) Any item $i$ can be assigned to a maximum one knapsack. It is possible that an item is not assigned to any knapsack
$$
H_{\text{single}}= \sum_{j=0}^{N-1} \left(\sum_{i=0}^{M-1} x_{i,j}\right)\cdot \left(\sum_{i=0}^{M-1} x_{i,j}-1 \right) \;.
$$

### 2) Ensure that the capacity of any knapsack is not exceeded. This is achieved by incorporated by introducing slack bits
$$
\text{H}_{\text{capacity}}=\sum_{i=0}^{M-1} \left[\left(\sum_{j=0}^{N-1} w_j\cdot x_{i,j}\right) + \left(\sum_{b=0}^{\lfloor\log{c_i}\rfloor} 2^{b}\cdot y_{i,b}\right) - c_i \right]^2 \;.
$$

### 3) Maximize the total value of assigned items in knapsacks

$$
H_{\text{obj}}= - \sum_{i=0}^{M-1} \sum_{j=0}^{N-1} v_{i,j} \cdot x_{i,j} \;.
$$

Finally, the complete QUBO is written as,

$$
H = \alpha\cdot H_{\text{single}} + \beta\cdot \text{H}_{\text{capacity}} + \gamma \cdot H_{\text{obj}} \;, \\
$$

where $\alpha,\beta $ are the penalty coefficients, and $\gamma$ is the objective weight. Note, $\alpha>0, \beta>0, \gamma >0$.


In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Problem Data

We first load the data for problem instances saved ine a json file `data/knapsack/qaoa_data.json`. We benchmark the studied algorithms on four different problem instances with increasing problem sizes. 

In [61]:
import sys
import json
import importlib
import numpy as np
import gurobipy as gp
import warnings
from qiskit_aer import AerSimulator
warnings.filterwarnings('ignore')

file      = 'data/knapsack/qaoa_data.json'
scenario  = 'scenario_2'

with open(file, 'r') as f:
    dataset = json.load(f)

article_reward = np.array(dataset[scenario]['article_reward'])
article_weight = np.array(dataset[scenario]['article_weight'])
knapsack_capacity = np.array(dataset[scenario]['knapsack_capacity'])
optimal = np.array(dataset[scenario]['optimal_solution'])
(number_of_knapsacks, number_of_articles) = article_reward.shape

## Classical Integer programming

As a first solver we present the evaluation code for Integer Programming, which models the multi-knapsack problem as a linear integer program and solves the model using `PuLp` solver. The code can also be run using the `Gurobi` solver.

In [62]:
import importlib
import QAOA.sources.KnapsackIP
importlib.reload(QAOA.sources.KnapsackIP)
from QAOA.sources.KnapsackIP import IPSolver

formulation = 'Binary'
solver      = None # None Gurobi

ip_solver=IPSolver(article_reward, article_weight, knapsack_capacity, formulation)
ip_solver.create_model()
status, solution, sol_y, consumed, objective_value = ip_solver.solve(solver)
objective_value, consumed, knapsack_capacity, article_weight

Variables Created
Single Knapsack done
Constraint with Slack Done
Objective Done


(12.0, {0: 5.0, 1: 5.0}, array([5, 5]), array([3, 3, 2, 2]))

## Standard QAOA

The code cell below provides the solver for the Quantum Approximate Optimization Algorithm (https://arxiv.org/abs/1411.4028). For the classical optimization, we utilize `SLSQP` solver. More details of the implementation can be found in the `QAOASolver` class provided in the source code.

In [63]:
import importlib
import QAOA.sources.KnapsackQAOA_Qiskit
importlib.reload(QAOA.sources.KnapsackQAOA_Qiskit)
from QAOA.sources.KnapsackQAOA_Qiskit import QAOASolver


total_iterations  = 1000
n_shots           = 10000
num_layers        = 3
formulation       = 'Binary'

init_params=(np.pi/2*np.random.rand(1,2)).tolist()[0]
single_penalty = int(2*np.max(article_reward))
capacity_penalty = int(2*np.max(article_reward))
objective_weight  = 1

qaoa = QAOASolver(article_reward, article_weight, knapsack_capacity, formulation,
                  single_penalty, capacity_penalty, objective_weight,num_layers)
                

all_solutions, samples, runtime, classical_iterations= qaoa.solve_qaoa_knapsack(init_params, total_iterations, num_layers, n_shots)

AttributeError: 'SparsePauliOp' object has no attribute 'primitive_strings'

## Warm-start QAOA

The next code cell provides the evaluation code for Warm-start QAOA (https://arxiv.org/abs/2009.10095). 

In [ ]:
import importlib
import numpy as np

import QAOA.sources.KnapsackWSQAOA_Qiskit
importlib.reload(QAOA.sources.KnapsackWSQAOA_Qiskit)
from QAOA.sources.KnapsackWSQAOA_Qiskit import WSQAOASolver

import warnings
warnings.filterwarnings('ignore')


total_iterations  = 1000
n_shots           = 10000
num_layers        = 3
formulation       = 'Binary'


init_params       = (np.pi/2*np.random.rand(1,2)).tolist()[0]
single_penalty    = int(2*np.max(article_reward))
capacity_penalty  = int(2*np.max(article_reward))
epsilon           = 0.25
mixer_hamiltonian = 'WS-Mixer'



init_params=(np.pi/2*np.random.rand(1,2)).tolist()[0]

formulation = 'Binary'
qaoa = WSQAOASolver(article_reward, article_weight, knapsack_capacity, formulation,
                  single_penalty, capacity_penalty, objective_weight,num_layers,
                  epsilon)
                
print('WS-QAOA in progress...')
print('Number of qubits', len(qaoa.variables))
all_solutions, samples, runtime, classical_iterations = qaoa.solve_qaoa_knapsack(mixer_hamiltonian, init_params, total_iterations, num_layers, n_shots)



Restricted license - for non-production use only - expires 2025-11-24
Set parameter NonConvex to value 2
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 0 rows, 14 columns and 0 nonzeros
Model fingerprint: 0x2b40e15c
Model has 60 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+02, 4e+02]
  QObjective range [2e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolved: 0 rows, 14 columns, 0 nonzeros
Presolved model has 60 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 6
 AA' NZ     : 1.400e+01
 Factor NZ  : 2.100e+01
 Factor Ops : 9.100e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective          

AttributeError: 'SparsePauliOp' object has no attribute 'primitive_strings'

## Warm-start QAOA with Pauli-X Hamiltonian

We also benchmark a variant of warm-start QAOA (WS-QAOA), where in, a Pauli-X mixer Hamiltonian is used instead of the proposed mixer Hamiltonian in https://arxiv.org/abs/2009.10095. The idea of this variation was to check the effect of the mixer Hamiltonian on the fianl solution, on top of warm-started initial state of the quantum circuit.

In [ ]:
import importlib
import numpy as np

import QAOA.sources.KnapsackWSQAOA_Qiskit
importlib.reload(QAOA.sources.KnapsackWSQAOA_Qiskit)
from QAOA.sources.KnapsackWSQAOA_Qiskit import WSQAOASolver

import warnings
warnings.filterwarnings('ignore')


total_iterations  = 1000
n_shots           = 10000
num_layers        = 3
formulation       = 'Binary'

init_params       = (np.pi/2*np.random.rand(1,2)).tolist()[0]
single_penalty    = int(2*np.max(article_reward))
capacity_penalty  = int(2*np.max(article_reward))
epsilon           = 0.25
mixer_hamiltonian = 'Pauli-X'

init_params=(np.pi/2*np.random.rand(1,2)).tolist()[0]

formulation = 'Binary'

qaoa = WSQAOASolver(article_reward, article_weight, knapsack_capacity, formulation,
                  single_penalty, capacity_penalty, objective_weight,num_layers,
                  epsilon)
print('WS-QAOA in progress...')                
print('Number of qubits', len(qaoa.variables))
all_solutions, samples, runtime, classical_iterations = qaoa.solve_qaoa_knapsack(mixer_hamiltonian, init_params, total_iterations, num_layers, n_shots)



Set parameter NonConvex to value 2
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 0 rows, 14 columns and 0 nonzeros
Model fingerprint: 0x2b40e15c
Model has 60 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+02, 4e+02]
  QObjective range [2e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolved: 0 rows, 14 columns, 0 nonzeros
Presolved model has 60 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 6
 AA' NZ     : 1.400e+01
 Factor NZ  : 2.100e+01
 Factor Ops : 9.100e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

AttributeError: 'SparsePauliOp' object has no attribute 'primitive_strings'

# Variational Quantum Eigensolver (VQE)

Another circuit model algorithm which we benchmark the Variational Quantum Eigensolver. The circuit ansatz used in our implementation has been adopted from https://arxiv.org/pdf/2102.05566.pdf. 

In [ ]:
import importlib

import VQE.VQE
importlib.reload(VQE.VQE)
from VQE.VQE import solve_VQE, VQE_circuit

from Data.QUBO_builder import build_QUBO

from qiskit import Aer, circuit
backend = Aer.get_backend("aer_simulator")

# build QUBO matrix for scnenario number 1
Q,offset = build_QUBO(scenario_no=1)

# solve QUBO problem with VQE
results = solve_VQE(backend=backend, Q=Q, offset=offset, num_layers=1)

VQE best solution:  000100001110
VQE best energy:  -9.0
Opt. iterations: 771 




In [ ]:
circuit = VQE_circuit(num_qubits=4, num_layers=2)
circuit.draw()

┌─────────┐ ░      ┌─────────┐     ┌─────────┐                     »
q_0: ┤ Ry(θ_0) ├─░───■──┤ Ry(θ_4) ├──■──┤ Ry(θ_6) ├─────────────────────»
     ├─────────┤ ░ ┌─┴─┐├─────────┤┌─┴─┐├─────────┤     ┌─────────┐     »
q_1: ┤ Ry(θ_1) ├─░─┤ X ├┤ Ry(θ_5) ├┤ X ├┤ Ry(θ_7) ├──■──┤ Ry(θ_8) ├──■──»
     ├─────────┤ ░ └───┘└─────────┘└───┘└─────────┘┌─┴─┐├─────────┤┌─┴─┐»
q_2: ┤ Ry(θ_2) ├─░─────────────────────────────────┤ X ├┤ Ry(θ_9) ├┤ X ├»
     ├─────────┤ ░                                 └───┘└─────────┘└───┘»
q_3: ┤ Ry(θ_3) ├─░──────────────────────────────────────────────────────»
     └─────────┘ ░                                                      »
c: 4/═══════════════════════════════════════════════════════════════════»
                                                                        »
«                                                    ░      ┌──────────┐     »
«q_0: ───────────────────────────────────────────────░───■──┤ Ry(θ_16) ├──■──»
«     ┌──────────┐                                   ░ ┌─┴─┐├──────────┤┌─┴─┐»
«q_1: ┤ Ry(θ_10) ├───────────────────────────────────░─┤ X ├┤ Ry(θ_17) ├┤ X ├»
«     ├──────────┤     ┌──────────┐     ┌──────────┐ ░ └───┘└──────────┘└───┘»
«q_2: ┤ Ry(θ_11) ├──■──┤ Ry(θ_12) ├──■──┤ Ry(θ_14) ├─░───────────────────────»
«     └──────────┘┌─┴─┐├──────────┤┌─┴─┐├──────────┤ ░                       »
«q_3: ────────────┤ X ├┤ Ry(θ_13) ├┤ X ├┤ Ry(θ_15) ├─░───────────────────────»
«                 └───┘└──────────┘└───┘└──────────┘ ░                       »
«c: 4/═══════════════════════════════════════════════════════════════════════»
«                                                                            »
«     ┌──────────┐                                                        »
«q_0: ┤ Ry(θ_18) ├────────────────────────────────────────────────────────»
«     ├──────────┤     ┌──────────┐     ┌──────────┐                      »
«q_1: ┤ Ry(θ_19) ├──■──┤ Ry(θ_20) ├──■──┤ Ry(θ_22) ├──────────────────────»
«     └──────────┘┌─┴─┐├──────────┤┌─┴─┐├──────────┤     ┌──────────┐     »
«q_2: ────────────┤ X ├┤ Ry(θ_21) ├┤ X ├┤ Ry(θ_23) ├──■──┤ Ry(θ_24) ├──■──»
«                 └───┘└──────────┘└───┘└──────────┘┌─┴─┐├──────────┤┌─┴─┐»
«q_3: ──────────────────────────────────────────────┤ X ├┤ Ry(θ_25) ├┤ X ├»
«                                                   └───┘└──────────┘└───┘»
«c: 4/════════════════════════════════════════════════════════════════════»
«                                                                         »
«                  ░ 
«q_0: ─────────────░─
«                  ░ 
«q_1: ─────────────░─
«     ┌──────────┐ ░ 
«q_2: ┤ Ry(θ_26) ├─░─
«     ├──────────┤ ░ 
«q_3: ┤ Ry(θ_27) ├─░─
«     └──────────┘ ░ 
«c: 4/═══════════════
«

# Annealing

For the annealing part of the paper we used the open-source benchmarking framework QUARK. We refer the reader to the [source code](https://github.com/QUARK-framework/QUARK) and the official [documentation](https://quark-framework.readthedocs.io/en/latest/).

The following example is an MVP of how the knapsack problem is solved with a the simulated annealing approach described in the paper. Because the DWave devices were removed from AWS Braket in November 2022, some minor adjustments were made to some of the files (now labelled "..._without_DWave") but the original files have not been removed, so results can still be reproduced if the user have access to DWave devices.

In [ ]:
pip install inquirer

     ---------------------------------------- 0.0/58.4 kB ? eta -:--:--
     --------------------- ------------------ 30.7/58.4 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 58.4/58.4 kB 609.3 kB/s eta 0:00:00
     ---------------------------------------- 0.0/63.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/63.7 kB ? eta -:--:--
     -------------------------------------- - 61.4/63.7 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 63.7/63.7 kB 861.9 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\admin\appdata\local\programs\python\python311\lib\site-packages\covid_abs-0.0.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [ ]:
pip install seaborn

   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   -------- ------------------------------- 61.4/294.6 kB 3.2 MB/s eta 0:00:01
   --------- ----------------------------- 71.7/294.6 kB 975.2 kB/s eta 0:00:01
   ------------ --------------------------- 92.2/294.6 kB 1.0 MB/s eta 0:00:01
   --------------- ---------------------- 122.9/294.6 kB 717.5 kB/s eta 0:00:01
   ------------------ ------------------- 143.4/294.6 kB 708.1 kB/s eta 0:00:01
   ------------------ ------------------- 143.4/294.6 kB 708.1 kB/s eta 0:00:01
   ---------------------- --------------- 174.1/294.6 kB 551.6 kB/s eta 0:00:01
   -------------------------- ----------- 204.8/294.6 kB 591.4 kB/s eta 0:00:01
   ------------------------------ ------- 235.5/294.6 kB 533.8 kB/s eta 0:00:01
   --------------------------------- ---- 256.0/294.6 kB 524.0 kB/s eta 0:00:01
   ----------------------------------- -- 276.5/294.6 kB 532

DEPRECATION: Loading egg at c:\users\admin\appdata\local\programs\python\python311\lib\site-packages\covid_abs-0.0.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


   -------------------------------------- 294.6/294.6 kB 343.0 kB/s eta 0:00:00


In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.dirname("Annealing/QUARK/src/")))
sys.path.append(os.path.join(os.path.dirname("Annealing/QUARK/paper_config/")))
import yaml
import Annealing.QUARK.src.config as config
import Annealing.QUARK.src.main as main
import Annealing.QUARK.src.BenchmarkManager as BenchmarkManager
import Annealing.QUARK.src.applications.Application
import Annealing.QUARK.src.applications.Mapping
import Annealing.QUARK.src.applications.Knapsack
import Annealing.QUARK.src.applications.Knapsack.mappings.MultiKSQUBO
import Annealing.QUARK.src.devices.Device
import Annealing.QUARK.src.devices.SimulatedAnnealingSampler
import Annealing.QUARK.src.solvers.Solver
import Annealing.QUARK.src.solvers.Annealer_without_DWave

benchmark_manager = BenchmarkManager.BenchmarkManager()

f = open("Annealing/QUARK/paper_config/annealing_example.yml")
benchmark_config = yaml.load(f, Loader=yaml.FullLoader)
benchmark_manager.orchestrate_benchmark(benchmark_config)
df = benchmark_manager.load_results()
benchmark_manager.vizualize_results(df)


<Figure size 446.25x300 with 0 Axes>

<Figure size 446.25x300 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

<Figure size 395.153x300 with 0 Axes>

Now you can have a look at the results in [benchmark_runs](./benchmark_runs/).

# Calculation of circuit runtimes

In addition to the evaluation code we also provide an estimate of circuit runtimes on a real hardware, for standard QAOA and VQE.

In [ ]:
import json
import Runtimes.circ_runtime_evaluation
importlib.reload(Runtimes.circ_runtime_evaluation)
from Runtimes.circ_runtime_evaluation import *

scenarios = np.array([1,2,3,4])
num_layers = 1
objective_weight = 1
formulation      = 'Binary'

backend = FakeBrooklyn()

#averages for transpilation
comp_averages=20

#optimization level for transpilation
opt_level = 3 

algorithm='vqe' # 'qaoa', 'vqe'

file = 'data/knapsack/qaoa_data.json'
with open(file, 'r') as f:
    dataset = json.load(f)

# create object and calculate runtimes
calc_runtimes = circ_runtime_evaluation(dataset, scenarios, num_layers, objective_weight, formulation, comp_averages, opt_level, backend, algorithm)

runtimes_mean = calc_runtimes.runtimes_mean
print('Mean Circuit Runtimes for VQE')
print(f'Scenario \t runtime (millisecond) \t standard-deviation')
print('-----------------------------------------------------')
for i in range(len(runtimes_mean)):
    print(f'scenario_{i+1}\t {runtimes_mean[i][0]} \t {runtimes_mean[i][1]}')
print('\n\n')    



algorithm='qaoa' # 'qaoa', 'vqe'

# create object and calculate runtimes
calc_runtimes = circ_runtime_evaluation(dataset, scenarios, num_layers, objective_weight, formulation, comp_averages, opt_level, backend, algorithm)

runtimes_mean = calc_runtimes.runtimes_mean
print('Mean Circuit Runtimes for QAOA')
print(f'Scenario \t runtime (millisecond) \t standard-deviation')
print('-----------------------------------------------------')
for i in range(len(runtimes_mean)):
    print(f'scenario_{i+1}\t {runtimes_mean[i][0]} \t {runtimes_mean[i][1]}')    

NameError: name 'FakeBrooklyn' is not defined